In [5]:
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import arff
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
bitcoin_db_path = '../BitcoinHeist_Ransomware/dataset.arff'
bitcoin_data = arff.load(open(bitcoin_db_path,'r'))

bitcoin_columns = [c[0] for c in bitcoin_data['attributes']]
bitcoin_columns

bitcoin_df = pd.DataFrame(bitcoin_data['data'], columns=bitcoin_columns)
bitcoin_df.shape

# y_labels = bitcoin_df['label'] == 'white'
# bitcoin_df['y'] = y_labels.astype('int')

bitcoin_df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000,princetonLocky


In [2]:
## making duplicate row if dataset only has 1 row for given class
label_grp_df = bitcoin_df.groupby('label')
min_label_size = 2
test_size = 0.2
small_sized_labels = []
small_sized_labels_df = pd.DataFrame(columns=bitcoin_df.columns)
# y_df = pd.get_dummies(bitcoin_df.iloc[:,-1])
train_df = pd.DataFrame(columns=bitcoin_df.columns)
test_df = pd.DataFrame(columns=bitcoin_df.columns)

for grp_label, grp in label_grp_df:
    grp_size = len(grp)
    print(grp_label, grp_size)
    if grp_size < min_label_size:
        small_sized_labels.append(grp_label)
        #print(f'grp:{grp}')
        for row in grp.iterrows():
            #print('row',row[1])
            small_sized_labels_df = small_sized_labels_df.append(row[1], ignore_index=True)
            train_df = train_df.append(row[1], ignore_index=True)
            test_df = test_df.append(row[1], ignore_index=True)
    else:
        temp_train, temp_test = train_test_split(grp, test_size=0.2)
        print(f'\t temp_train size:{temp_train.shape}, temp_test size:{temp_test.shape}')
        train_df = pd.concat([train_df, temp_train], ignore_index=True)
        test_df = pd.concat([test_df, temp_test], ignore_index=True)
        
print(f'\nsmall sized labels:{small_sized_labels}')
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)
small_sized_labels_df

NameError: name 'bitcoin_df' is not defined

In [3]:
X_train = train_df.iloc[:, 1:9]
y_train = pd.get_dummies(train_df.iloc[:,-1])
X_test = test_df.iloc[:, 1:9]
y_test = pd.get_dummies(test_df.iloc[:,-1])

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

NameError: name 'train_df' is not defined

In [13]:
%%time
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.model_selection import cross_val_score

from scikit_obliquetree.HHCART import HouseHolderCART, HHCARTNode
from scikit_obliquetree.segmentor import MSE, MeanSegmentor, ABCMeta


# clf = BaggingClassifier(
#         HHCARTNode(ABCMeta(), MeanSegmentor(), max_depth=3),
#         n_estimators=100,
#         n_jobs=-1
# )

reg = BaggingRegressor(
    HouseHolderCART(MSE(), MeanSegmentor(), max_depth=3),
    n_estimators=30,
    n_jobs=-1,
)

Wall time: 0 ns


In [ ]:
%%time
print('CV Score', cross_val_score(reg, X_train, np.argmax(y_train.values, axis=1)))

In [ ]:
print()